In [1]:
from kiteconnect import KiteConnect
from datetime import datetime,timedelta
import pandas as pd
import plotly.graph_objects as go

In [4]:
kite = KiteConnect(api_key="3np0d05xqq02k905")
kite.set_access_token("GNyGiY8y10NG335riTWjXfGeuETCLB3t")

In [5]:
df_instruments=pd.DataFrame(kite.instruments())

In [6]:
def get_candle_fig(instrument_code,start,end,interval):
    df_historical=pd.DataFrame(kite.historical_data(instrument_code,start,end,interval))
    if len(df_historical)>0:
        fig = go.Figure(data=[go.Candlestick(x=df_historical['date'],
                        open=df_historical['open'],
                        high=df_historical['high'],
                        low=df_historical['low'],
                        close=df_historical['close'])])
        return (fig,df_historical)
    else:
        return (None,pd.DataFrame())

In [7]:
def get_candle(instrument_code,start,end,interval):
    df_historical=pd.DataFrame(kite.historical_data(instrument_code,start,end,interval))
    if len(df_historical)>0:
        return (df_historical)
    else:
        return (pd.DataFrame())

In [8]:
def get_candle_signal(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:'green' if x>0 else 'red')
                try:
                    candle_ratio= ((df_historical['candle_type'].value_counts()['green'])/(df_historical['candle_type'].count()))
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return ((sum(df_summary['candle_ratio'])*2)/sample_size)
    except:
        return 0

In [25]:
def get_candle_signal_2(instrument_code,sample_size):
    try:
        df_summary=pd.DataFrame()
        for i in range(sample_size):
            start=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 09:00:00"
            end=((datetime.now()+timedelta(days=-i)).strftime("%Y-%m-%d"))+" 16:00:00"
            interval="5minute"
            df_historical=get_candle(instrument_code,start,end,interval)
            
            if len(df_historical)>0:
                df_historical['change']=df_historical['close']-df_historical['open']
                df_historical['candle_type']=df_historical['change'].apply(lambda x:'green' if x>0 else 'red')
                try:
                    candle_ratio= ((df_historical['candle_type'].value_counts()['green'])/(df_historical['candle_type'].count()))
                except:
                    candle_ratio=0
                df_summary_dict={'day':end,
                                'candle_ratio':candle_ratio}
                df_temp=pd.DataFrame([df_summary_dict])
                df_summary=pd.concat([df_summary,df_temp])
        return (len(df_summary[df_summary['candle_ratio']>0.5])/len(df_summary))
    except:
        return 0

In [14]:
df_instruments1=df_instruments[(df_instruments['instrument_type']=='EQ')&(df_instruments['name']!="")&(df_instruments['segment']=="NSE")].head(10)
df_instruments1['signal_1']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal(x,7))
df_instruments1['signal_2']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_2(x,7))

In [24]:
df_instruments1['signal_2']=df_instruments1['instrument_token'].apply(lambda x: get_candle_signal_2(x,7))

In [26]:
df_instruments1

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,signal_1,signal_2
69857,4331777,16921,20MICRONS,20 MICRONS,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.480013,0.00
69858,1984001,7750,21STCENMGM-BE,21ST CENTURY MGMT SERVICE,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.052381,0.00
69859,3343617,13061,360ONE,360 ONE WAM,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.549048,0.25
69911,1595393,6232,3IINFOLTD,3I INFOTECH,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.470476,0.25
69912,121345,474,3MINDIA,3M INDIA,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.503333,0.25
69913,2316289,9048,3PLAND-BE,3P LAND HOLDINGS,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.117451,0.00
69996,113921,445,5PAISA,5PAISA CAPITAL,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.446984,0.25
70052,3038209,11868,63MOONS,63 MOONS TECHNOLOGIES,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.247722,0.00
72733,5352961,20910,A2ZINFRA-BE,A2Z INFRA ENGINEERING,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.000000,0.00
72734,3210753,12542,AAATECH-BE,AAA TECHNOLOGIES,0.0,,0.0,0.05,1,EQ,NSE,NSE,0.159841,0.00


In [16]:
len(df_instruments[(df_instruments['instrument_type']=='EQ')&(df_instruments['name']!="")&(df_instruments['segment']=="NSE")])

2440

In [ ]:

# Place an order
order_id = kite.place_order(tradingsymbol="INFY",
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                            quantity=1,
                            variety=kite.VARIETY_AMO,
                            order_type=kite.ORDER_TYPE_MARKET,
                            product=kite.PRODUCT_CNC,
                            validity=kite.VALIDITY_DAY)



# Fetch all orders
kite.orders()

# Get instruments
kite.instruments()

# Place an mutual fund order
kite.place_mf_order(
    tradingsymbol="INF090I01239",
    transaction_type=kite.TRANSACTION_TYPE_BUY,
    amount=5000,
    tag="mytag"
)

# Cancel a mutual fund order
kite.cancel_mf_order(order_id="order_id")

# Get mutual fund instruments
kite.mf_instruments()